<a href="https://colab.research.google.com/github/mannsgreg/OPENMM/blob/main/3Dto3D(100saves%20and%20zip).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title **Install Conda Colab**
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [1]:
%%capture
!pip install git+https://github.com/pablo-arantes/biopandas
!mamba install openmmforcefields -c conda-forge -y
!pip install openMiChroM

In [3]:
from OpenMiChroM.ChromDynamics import MiChroM
from OpenMiChroM.CndbTools import cndbTools
import numpy as np

In [4]:
sim = MiChroM(temperature=1.0, time_step=0.01)

    ***************************************************************************************     
     **** **** *** *** *** *** *** *** OpenMiChroM-1.0.7 *** *** *** *** *** *** **** ****      

         OpenMiChroM is a Python library for performing chromatin dynamics simulations.         
                            OpenMiChroM uses the OpenMM Python API,                             
                employing the MiChroM (Minimal Chromatin Model) energy function.                
      The chromatin dynamics simulations generate an ensemble of 3D chromosomal structures      
      that are consistent with experimental Hi-C maps, also allows simulations of a single      
                 or multiple chromosome chain using High-Performance Computing                  
                            in different platforms (GPUs and CPUs).                             
         OpenMiChroM documentation is available at https://open-michrom.readthedocs.io          

         OpenMiChroM is desc

In [15]:
sim.setup(platform="cuda")

In [16]:
sim.saveFolder('101_ep5sig.1')

In [8]:
chr2 = sim.createSpringSpiral(ChromSeq='/content/chr2_compartments.txt', isRing=False)

In [9]:
sim.loadStructure(chr2, center=True)

In [10]:
sim.saveStructure(mode='ndb')

In [11]:
sim.addFENEBonds(kfb=30.0)
sim.addAngles(ka=2.0)
sim.addRepulsiveSoftCore(Ecut=4.0)

In [12]:
def AM_bond(sim, pair_list, depth, sigma=1):
    R"""
    Internal function that inits FM bond force.
    """
    bondforceGr = sim.mm.CustomBondForce(f'-depth_w*exp((r-r0)^2/(-2.0*sigma^2))')
    bondforceGr.addGlobalParameter('depth_w', depth)
    bondforceGr.addPerBondParameter('r0')
    bondforceGr.addPerBondParameter('sigma')

    sim.forceDict["AM"] = bondforceGr

    for k  in range(len(pair_list)):
        i, j, ro = pair_list[k]
        sim.forceDict["AM"].addBond(int(i), int(j), [ro,np.sqrt(j-i)*sigma])

In [13]:
pair_list=np.loadtxt('/content/pairs_chr2_h1.txt')
pair_list[:,2]=np.round(pair_list[:,2],4)
print('Number of constrains:',len(pair_list))

Number of constrains: 436645


In [17]:
AM_bond(sim,pair_list,5,sigma=.1)

In [18]:
sim.addFlatBottomHarmonic(kr=5*10**-3, n_rad=15.0)

In [19]:
block = 3*10**2
n_blocks = 2*10**3

In [20]:
for _ in range(n_blocks):
    sim.runSimBlock(block, increment=False)

Number of exceptions: 0
adding force  AM 0
adding force  FlatBottomHarmonic 1
Positions... 
 loaded!
potential energy is 154.058887
bl=0 pos[1]=[373.8 -20.0 -4.5] dr=1.64 t=3.0ps kin=2.02 pot=153.41 Rg=262.291 SPS=2632 
bl=0 pos[1]=[370.7 -20.2 -4.8] dr=2.06 t=6.0ps kin=3.12 pot=151.74 Rg=260.956 SPS=2900 
bl=0 pos[1]=[366.4 -21.1 -5.2] dr=2.51 t=9.0ps kin=4.21 pot=149.34 Rg=259.018 SPS=3491 
bl=0 (i) pos[1]=[362.5 -21.7 -5.2] dr=2.86 t=12.0ps kin=5.20 pot=146.43 Rg=256.633 SPS=5627 
bl=0 pos[1]=[361.4 -21.3 -5.1] dr=1.64 t=15.0ps kin=2.00 pot=145.82 Rg=256.147 SPS=5667 
bl=0 pos[1]=[358.1 -21.0 -5.8] dr=2.02 t=18.0ps kin=2.97 pot=144.25 Rg=254.857 SPS=5680 
bl=0 pos[1]=[353.6 -20.6 -6.5] dr=2.45 t=21.0ps kin=4.06 pot=141.99 Rg=252.971 SPS=5737 
bl=0 (i) pos[1]=[349.1 -20.4 -7.8] dr=2.81 t=24.0ps kin=5.02 pot=139.22 Rg=250.644 SPS=5729 
bl=0 pos[1]=[345.9 -19.3 -7.1] dr=1.62 t=27.0ps kin=1.95 pot=138.62 Rg=250.158 SPS=5735 
bl=0 pos[1]=[342.0 -18.8 -5.8] dr=2.00 t=30.0ps kin=2.90 pot=1

In [21]:
print(sim.chromRG())
sim.saveStructure(mode='ndb')

29.466106


In [22]:
sim.saveStructure(mode='gro')
sim.saveStructure(mode='pdb')

In [23]:
sim.removeFlatBottomHarmonic()

Removed FlatBottomHarmonic from the system!


In [24]:
sim.forceDict.keys()

dict_keys(['AM'])

In [25]:
sim.initStorage(filename="traj_chr10")

In [26]:
block = 5*10**2
n_blocks = 5*10**3

In [27]:
for ok in range(n_blocks):
    sim.runSimBlock(block, increment=True)
    #sim.saveStructure()
    if (ok) % 5 == 0:
       sim.saveStructure(filename=f"structure_{ok}.pdb",mode='pdb')

Streaming output truncated to the last 5000 lines.
bl=1718 pos[1]=[-5.1 -6.1 -2.7] dr=2.29 t=14590.0ps kin=1.55 pot=-450.43 Rg=206.671 SPS=5197 
bl=1719 pos[1]=[-5.2 -6.0 -2.9] dr=2.29 t=14595.0ps kin=1.51 pot=-450.41 Rg=206.734 SPS=5202 
bl=1720 pos[1]=[-5.3 -6.2 -3.0] dr=2.29 t=14600.0ps kin=1.51 pot=-450.41 Rg=206.805 SPS=5154 
bl=1721 pos[1]=[-5.3 -5.8 -3.2] dr=2.28 t=14605.0ps kin=1.48 pot=-450.41 Rg=206.885 SPS=5136 
bl=1722 pos[1]=[-5.2 -5.9 -3.2] dr=2.27 t=14610.0ps kin=1.50 pot=-450.42 Rg=206.971 SPS=5294 
bl=1723 pos[1]=[-5.1 -5.7 -3.2] dr=2.28 t=14615.0ps kin=1.50 pot=-450.42 Rg=207.027 SPS=5081 
bl=1724 pos[1]=[-5.1 -5.9 -3.1] dr=2.25 t=14620.0ps kin=1.48 pot=-450.42 Rg=207.064 SPS=5175 
bl=1725 pos[1]=[-5.2 -5.9 -3.2] dr=2.27 t=14625.0ps kin=1.51 pot=-450.41 Rg=207.075 SPS=5177 
bl=1726 pos[1]=[-5.3 -6.0 -3.1] dr=2.28 t=14630.0ps kin=1.50 pot=-450.42 Rg=207.081 SPS=5196 
bl=1727 pos[1]=[-5.3 -6.0 -3.2] dr=2.29 t=14635.0ps kin=1.51 pot=-450.42 Rg=207.090 SPS=5286 
bl=1728 p

In [28]:
sim.storage[0].close()

In [29]:
sim.saveStructure(mode='pdb')

In [30]:
import shutil
import os
from google.colab import files

# Specify the folder path you want to save
folder_path = '/content/101_ep5sig.1'

# Specify the destination path for the zip file
zip_path = '/content/101_ep5sig.1'

# Create a zip file of the folder
shutil.make_archive(zip_path, 'zip', folder_path)

# Download the zip file
#files.download(zip_path)


'/content/101_ep5sig.1.zip'